In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

import xgboost as xgb
import pickle

In [3]:
profile = pd.read_csv("data/profile.csv")
portfolio = pd.read_csv("data/portfolio-dummy.csv")
train_set = pd.read_csv("data/train.csv")
test_set = pd.read_csv("data/test.csv")
train_transactions = pd.read_csv("data/train_transactions.csv")
test_transactions = pd.read_csv("data/test_transactions.csv")

In [12]:
def encode_dataset(df, transactions, shuffle=False):
    df = df.merge(profile, left_on='person', right_on='id') \
        .merge(portfolio, left_on='offer_id', right_on="id") \
        .merge(transactions, left_on='person', right_on='person')
    
    df = df.groupby("person").mean()
#     if shuffle:
#         df = df.sample(frac=1)
#     df.index = df[['person', 'offer_id']]
    
#     y = df['offer viewed']
#     df = df.drop(columns=['person', 'offer_id', 'id_x', 'id_y', 'index', 'offer viewed'])
    return df

X_train = encode_dataset(train_set, train_transactions)
X_train

,offer viewed,age,income,income_na,gender_F,gender_M,gender_O,age_na,member_since,index,...,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational,value
person,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,1.000000,33.000000,72000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,5.333333,...,1.666667,4.000000,1.0,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,30.73
00116118485d4dfda04fdbaba9a87b5c,1.000000,54.393524,65404.991568,1.0,0.0,0.0,0.0,1.0,2018.0,8.000000,...,5.000000,5.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.09
0011e0d4e6b944f998e987f904e8c1e5,1.000000,40.000000,57000.000000,0.0,0.0,0.0,1.0,0.0,2018.0,2.250000,...,6.750000,6.000000,1.0,0.750000,0.500000,0.750000,0.000000,0.500000,0.500000,25.42
0020c2b971eb4e9188eac86d93036a77,0.666667,59.000000,90000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,6.666667,...,10.000000,7.333333,1.0,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,98.33
0020ccbbb6d84e358d3414a3ff76cffd,1.000000,24.000000,60000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,4.666667,...,4.000000,5.000000,1.0,1.000000,1.000000,0.666667,0.333333,0.333333,0.333333,136.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff3ba4757bd42088c044ca26d73817a,0.500000,69.000000,83000.000000,0.0,1.0,0.0,0.0,0.0,2015.0,5.500000,...,6.250000,6.750000,1.0,1.000000,0.500000,0.750000,0.250000,0.500000,0.250000,148.84
fff7576017104bcc8677a8d63322b5e1,1.000000,71.000000,73000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,6.666667,...,10.000000,7.333333,1.0,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,16.05
fff8957ea8b240a6b5e634b6ee8eafcf,1.000000,71.000000,56000.000000,0.0,0.0,1.0,0.0,0.0,2018.0,9.000000,...,10.000000,10.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,11.26


In [13]:
test_set

,person,offer_id,offer viewed
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0.0
1,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1.0
2,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,1.0
3,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0
4,0020c2b971eb4e9188eac86d93036a77,5a8bc65990b245e5a138643cd4eb9837,1.0
...,...,...,...
24516,fff8957ea8b240a6b5e634b6ee8eafcf,3f207df678b143eea3cee63160fa8bed,0.0
24517,fff8957ea8b240a6b5e634b6ee8eafcf,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0
24518,fffad4f4828548d1b5583907f2e9906b,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0
24519,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,1.0


In [20]:

total_set

,person,age,income,income_na,gender_F,gender_M,gender_O,age_na,member_since,index,...,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational,value,offer_id
0,0009655768c64bdeb2e877511632db8f,33.000000,72000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,5.333333,...,4.000000,1.0,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,30.73,2906b810c7d4411798c6938adc9daaa5
1,0009655768c64bdeb2e877511632db8f,33.000000,72000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,5.333333,...,4.000000,1.0,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,30.73,fafdcd668e3743c1bb461111dcafc2a4
2,00116118485d4dfda04fdbaba9a87b5c,54.393524,65404.991568,1.0,0.0,0.0,0.0,1.0,2018.0,8.000000,...,5.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.09,f19421c1d4aa40978ebb69ca19b0e20d
3,0011e0d4e6b944f998e987f904e8c1e5,40.000000,57000.000000,0.0,0.0,0.0,1.0,0.0,2018.0,2.250000,...,6.000000,1.0,0.750000,0.500000,0.750000,0.000000,0.500000,0.500000,25.42,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,0020c2b971eb4e9188eac86d93036a77,59.000000,90000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,6.666667,...,7.333333,1.0,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,98.33,5a8bc65990b245e5a138643cd4eb9837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23077,fff8957ea8b240a6b5e634b6ee8eafcf,71.000000,56000.000000,0.0,0.0,1.0,0.0,0.0,2018.0,9.000000,...,10.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,11.26,3f207df678b143eea3cee63160fa8bed
23078,fff8957ea8b240a6b5e634b6ee8eafcf,71.000000,56000.000000,0.0,0.0,1.0,0.0,0.0,2018.0,9.000000,...,10.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,11.26,4d5c57ea9a6940dd891ad53e9dbe8da0
23079,fffad4f4828548d1b5583907f2e9906b,34.000000,34000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,6.500000,...,4.000000,1.0,1.000000,1.000000,0.500000,0.500000,0.000000,0.500000,50.64,9b98b8c7a33c4b65b9aebfe6a799e6d9
23080,ffff82501cea40309d5fdd7edcca4a07,45.000000,62000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,3.666667,...,9.000000,1.0,0.666667,0.333333,1.000000,0.000000,1.000000,0.000000,175.02,2906b810c7d4411798c6938adc9daaa5


In [21]:
total_set_y

0        0.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
23077    0.0
23078    1.0
23079    1.0
23080    1.0
23081    1.0
Name: offer viewed_y, Length: 23082, dtype: float64

In [32]:
def create_youtube_rec_set(train_set, test_set):
    total_set = train_set.merge(test_set, left_on='person', right_on='person')
    total_set.index = total_set[['person', 'offer_id']]
    total_set_y = total_set['offer viewed_y']
    total_set = total_set.drop(columns=['offer viewed_x', 'offer viewed_y', 'person', 'offer_id'])
    
    return total_set, total_set_y 

X_total, y_total = create_youtube_rec_set(
    encode_dataset(train_set, train_transactions),
    test_set
)

In [33]:
X_total

,age,income,income_na,gender_F,gender_M,gender_O,age_na,member_since,index,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational,value
"(0009655768c64bdeb2e877511632db8f, 2906b810c7d4411798c6938adc9daaa5)",33.000000,72000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,5.333333,1.666667,1.666667,4.000000,1.0,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,30.73
"(0009655768c64bdeb2e877511632db8f, fafdcd668e3743c1bb461111dcafc2a4)",33.000000,72000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,5.333333,1.666667,1.666667,4.000000,1.0,1.000000,0.666667,0.666667,0.333333,0.000000,0.666667,30.73
"(00116118485d4dfda04fdbaba9a87b5c, f19421c1d4aa40978ebb69ca19b0e20d)",54.393524,65404.991568,1.0,0.0,0.0,0.0,1.0,2018.0,8.000000,5.000000,5.000000,5.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.09
"(0011e0d4e6b944f998e987f904e8c1e5, 9b98b8c7a33c4b65b9aebfe6a799e6d9)",40.000000,57000.000000,0.0,0.0,0.0,1.0,0.0,2018.0,2.250000,2.000000,6.750000,6.000000,1.0,0.750000,0.500000,0.750000,0.000000,0.500000,0.500000,25.42
"(0020c2b971eb4e9188eac86d93036a77, 5a8bc65990b245e5a138643cd4eb9837)",59.000000,90000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,6.666667,7.333333,10.000000,7.333333,1.0,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,98.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fff8957ea8b240a6b5e634b6ee8eafcf, 3f207df678b143eea3cee63160fa8bed)",71.000000,56000.000000,0.0,0.0,1.0,0.0,0.0,2018.0,9.000000,2.000000,10.000000,10.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,11.26
"(fff8957ea8b240a6b5e634b6ee8eafcf, 4d5c57ea9a6940dd891ad53e9dbe8da0)",71.000000,56000.000000,0.0,0.0,1.0,0.0,0.0,2018.0,9.000000,2.000000,10.000000,10.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,11.26
"(fffad4f4828548d1b5583907f2e9906b, 9b98b8c7a33c4b65b9aebfe6a799e6d9)",34.000000,34000.000000,0.0,0.0,1.0,0.0,0.0,2017.0,6.500000,2.500000,2.500000,4.000000,1.0,1.000000,1.000000,0.500000,0.500000,0.000000,0.500000,50.64
"(ffff82501cea40309d5fdd7edcca4a07, 2906b810c7d4411798c6938adc9daaa5)",45.000000,62000.000000,0.0,1.0,0.0,0.0,0.0,2016.0,3.666667,3.000000,13.333333,9.000000,1.0,0.666667,0.333333,1.000000,0.000000,1.000000,0.000000,175.02


In [34]:
y_total

(0009655768c64bdeb2e877511632db8f, 2906b810c7d4411798c6938adc9daaa5)    0.0
(0009655768c64bdeb2e877511632db8f, fafdcd668e3743c1bb461111dcafc2a4)    1.0
(00116118485d4dfda04fdbaba9a87b5c, f19421c1d4aa40978ebb69ca19b0e20d)    1.0
(0011e0d4e6b944f998e987f904e8c1e5, 9b98b8c7a33c4b65b9aebfe6a799e6d9)    1.0
(0020c2b971eb4e9188eac86d93036a77, 5a8bc65990b245e5a138643cd4eb9837)    1.0
                                                                       ... 
(fff8957ea8b240a6b5e634b6ee8eafcf, 3f207df678b143eea3cee63160fa8bed)    0.0
(fff8957ea8b240a6b5e634b6ee8eafcf, 4d5c57ea9a6940dd891ad53e9dbe8da0)    1.0
(fffad4f4828548d1b5583907f2e9906b, 9b98b8c7a33c4b65b9aebfe6a799e6d9)    1.0
(ffff82501cea40309d5fdd7edcca4a07, 2906b810c7d4411798c6938adc9daaa5)    1.0
(ffff82501cea40309d5fdd7edcca4a07, 9b98b8c7a33c4b65b9aebfe6a799e6d9)    1.0
Name: offer viewed_y, Length: 23082, dtype: float64

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.5, random_state=53, shuffle=True)

In [37]:
X_train.shape[0], X_test.shape[0]

(11541, 11541)

In [44]:
def get_xgb():
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('xgb', xgb.XGBClassifier(
                silent=False,
                objective='binary:logistic',
                booster='gbtree'
            ))
        ],
        verbose = True
    )
    
    return pipe

def get_xgb_gcv():
    
    param_grid = {
        'xgb__booster' :['gbtree'],
        'xgb__silent':[True],
        'xgb__max_depth':[5,6,8],
        'xgb__min_child_weight':[1,3,5],
        'xgb__gamma':[0,1,2,3],
        'xgb__nthread':[4],
        'xgb__colsample_bytree':[0.5,0.8],
        'xgb__colsample_bylevel':[0.9],
        'xgb__n_estimators':[50],
        'xgb__objective':['binary:logistic'],
        'xgb__random_state':[2]
    }
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('xgb', xgb.XGBClassifier())
        ],
        verbose = True
    )
    gcv=GridSearchCV(pipe, param_grid=param_grid, scoring='f1', n_jobs=1, verbose=1)
    
    return gcv

In [45]:
def train_and_test(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print("Report for", name)
    print("Train Report>")
    print("Train Accuracy:", model.score(X_train, y_train))
    y_pred = model.predict(X_train)
    print(classification_report(y_train, y_pred))
    
    print("Test Report>")
    print("Test Accuracy:", model.score(X_test, y_test))
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

In [46]:
%%time
non_opt_model = get_xgb()
train_and_test("xgb", non_opt_model, X_train, y_train, X_test, y_test)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.7s
Report for xgb
Train Report>
Train Accuracy: 0.8410016463044797
              precision    recall  f1-score   support

         0.0       0.92      0.40      0.56      2884
         1.0       0.83      0.99      0.90      8657

    accuracy                           0.84     11541
   macro avg       0.88      0.69      0.73     11541
weighted avg       0.85      0.84      0.82     11541

Test Report>
Test Accuracy: 0.7219478381422754
              precision    recall  f

In [47]:
%%time
model = get_xgb_gcv()
train_and_test("xgb-gcv", model, X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] .......

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:15:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificati

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.3s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:16:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.3s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:16:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificati

...

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificati

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:22] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificati

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.3s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:27] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.3s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[10:17:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificati

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  1.6min finished


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.2s
Report for xgb-gcv
Train Report>
Train Accuracy: 0.8644110401689206
              precision    recall  f1-score   support

         0.0       0.80      0.09      0.15      2884
         1.0       0.77      0.99      0.86      8657

    accuracy                           0.77     11541
   macro avg       0.78      0.54      0.51     11541
weighted avg       0.78      0.77      0.69     11541

Test Report>
Test Accuracy: 0.8508780732563974
              precision    recall  f1-score   support

         0.0       0.32      0.03      0.06      2874
         1.0       0.75      0.98      0.85      8667

    accuracy                           0.74     11541
   macro avg       0.54      0.50      0.45     11541
weighted avg       0.65      0.74      0.65     11541

Wall time: 1min 38s
